In [3]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace, ItemHelpers, MessageOutputItem
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings
from data_models.flight_data_models import FlightSearchResults, BestFlight, AirportInfo, Flight, Layover, OtherFlight, PriceInsights, AirportLocation, Airports
from data_models.yelp_data_models import Category, Business, YelpSearchResults
from travel_agents import question_rewrite_agent, flight_agent
# from functions import flight_search,yelp_search, hotels_search
from pprint import pprint
from data_models.hotel_data_models import HotelSearchResponse, SearchInformation, Brand, BrandChild, Property, GPSCoordinates, ReviewsBreakdown, Rating, Price, Rate, NearbyPlace, Transportation
from data_models.first_question_output_format import FinalModel
import datetime
from tools import flight_search, yelp_search,hotels_search


In [4]:
travel_agent = Agent(
    name = "Travel assistant", 
    instructions= """You are a talented and experienced travela assistant. Your task it to give informantion about traing ticket, hotels, 
    restaurants, basically give information about steps of the travel.""",
    model='gpt-4o-mini',
    # tools = [flight_agent,hotel_agent,yelp_agent]
)

question_rewrite_agent = Agent(
    name = "Customer Help",
    instructions= f"""
Today : {datetime.datetime.now().date()}

You are a helpful help desk agent at the travel company. Basically, your task it to understand the customers request, needs, demands. 
I want to you to learn and fill necessary fields to offer a reasonable travel to our customer. 

Try to extract the search terms from the question
    
""",
    model = 'gpt-4o-mini',
    # tools=[yelp_search],
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2),
    output_type = FinalModel,
    tools=[flight_search, yelp_search,hotels_search]
)

In [9]:
ank = "ankara(ESB)"
ist = "istanbul(SAW)"
amst = "amsterdam(AMS)"
pekin = "Peking(PEK)"

start_point = ank 
dest_point = amst

main_question = f"""I want to travel from {start_point} to {dest_point}. The outbound date is 2025-05-05 and the return date will be 2025-05-11. 
I also need a hotel to stay. I do not like smell of cigarette. If possible, I want to see the rooms where smoking is not allowed. 
In addition to that, I like eating sea food. I guess {dest_point} is very rich city interms of sea restaurants
for me show cheapest flight tickets, sea food restaurants, cheapest hotels
""".strip()


In [10]:
question_rewrite_agent_output = Runner.run_sync(question_rewrite_agent,main_question)

In [ ]:
question_rewrite_agent_output.final_output

FlightOutput(departure_id='ESB', arrival_id='AMS', outbound_data='2025-05-05', return_data='2025-05-11')

In [13]:
dir(question_rewrite_agent_output)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_last_agent',
 'final_output',
 'final_output_as',
 'input',
 'input_guardrail_results',
 'last_agent',
 'new_items',
 'output_guardrail_results',
 'raw_responses',
 'to_input_list']

In [15]:
question_rewrite_agent_output.new_items

[ToolCallItem(agent=Agent(name='Customer Help', instructions='\nToday : 2025-04-16\n\nYou are a helpful help desk agent at the travel company. Basically, your task it to understand the customers request, needs, demands. \nI want to you to learn and fill necessary fields to offer a reasonable travel to our customer. \n\nTry to extract the search terms from the question\n\n', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=0.0, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=8192, reasoning=None, metadata=None, store=None), tools=[FunctionTool(name='flight_search', description='Google flight information from SERPAPI. Returns json structure', params_json_schema={'properties': {'departure_id': {'title': 'Departure Id', 'type': 'string'}, 'arrival_id': {'title': 'Arrival Id', 'type': 'string'}, 'outbound_data': {'title': 'Outbound Data', 'type': 'string'}, 

In [20]:
question_rewrite_agent_output.new_items[3]

ToolCallOutputItem(agent=Agent(name='Customer Help', instructions='\nToday : 2025-04-16\n\nYou are a helpful help desk agent at the travel company. Basically, your task it to understand the customers request, needs, demands. \nI want to you to learn and fill necessary fields to offer a reasonable travel to our customer. \n\nTry to extract the search terms from the question\n\n', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=0.0, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=8192, reasoning=None, metadata=None, store=None), tools=[FunctionTool(name='flight_search', description='Google flight information from SERPAPI. Returns json structure', params_json_schema={'properties': {'departure_id': {'title': 'Departure Id', 'type': 'string'}, 'arrival_id': {'title': 'Arrival Id', 'type': 'string'}, 'outbound_data': {'title': 'Outbound Data', 'type': 'strin